## 4) Correlation between EEG features showing significant group differences

- The EEG features which showed at least one variable (i.e., electrode, brain region or microstate parameter) with a significant group effect were pairwise correlated
- If more than one variable of an EEG feature was significant after correcting for multiple comparisons, the variable showing the largest effect size was selected for the correlation analysis

Gordillo, da Cruz, Moreno, Garobbio, Herzog

In [ ]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
from scipy.stats import spearmanr

In [ ]:
main_dir = os.getcwd()
np.random.seed(234)

In [ ]:
# define data and results directories
data_dir = os.path.join(main_dir, 'data')
results_dir = os.path.join(main_dir, 'results')
os.chdir(main_dir)

# eeg features
path_eeg_csv = os.path.join(data_dir, 'csv_data')
eeg_features = list(filter(lambda x: '.csv' in x, os.listdir(path_eeg_csv)))
neeg = len(eeg_features)

# load mwu results
results_3_dir = os.path.join(results_dir, '3_group_comparison_results')
data_mwu = pd.read_csv(os.path.join(results_3_dir, '3_mwu_r.csv'), index_col=0)

In [ ]:
# Select the EEG features which showed a significant effect
significant_eeg = np.where(data_mwu['pvalues'] < 0.05)[0]

# concatenate features showing the biggest effect sizes
for ifeat in range(len(significant_eeg)):
    
    name_feat = data_mwu.index[significant_eeg[ifeat]]
    ch_feat = data_mwu.iloc[significant_eeg[ifeat]]['selected_ch']
    eeg_feat = pd.read_csv(os.path.join(path_eeg_csv,name_feat), index_col=0)
    eeg_feat = eeg_feat.iloc[:, [1,int(ch_feat + 2)]]
    
    if ifeat == 0:
        local_concat = eeg_feat.copy(deep=True)
    else:
        eeg_feat = eeg_feat.drop(['Group'], axis=1)
        local_concat = pd.concat([local_concat, eeg_feat], axis=1)
        
# group 1 data (young adults)
y_data = local_concat.loc[local_concat['Group'] == 1]
y_data = y_data.drop(['Group'], axis=1)
# correlate variables
y_correlation, y_pvalue = spearmanr(y_data)

# group 2 data (older adults)
o_data = local_concat.loc[local_concat['Group'] == 2]
o_data = o_data.drop(['Group'], axis=1)
# correlate variables
o_correlation, o_pvalue = spearmanr(o_data)

In [ ]:
# save_data
results_4_dir = os.path.join(results_dir, '4_correlations_groups_results')
eeg_feats = data_mwu.index[significant_eeg]
save_index = [feature.replace('.csv','') for feature in eeg_feats]

In [ ]:
# group 1
# correlation values
y_correlation_df = pd.DataFrame(data=y_correlation, index=save_index,
                                columns=save_index)
y_correlation_df.to_csv(os.path.join(results_4_dir, '4_correlation_eeg_y.csv'))

# p values
y_pvalue_df = pd.DataFrame(data=y_pvalue, index=save_index,
                           columns=save_index)
y_pvalue_df.to_csv(os.path.join(results_4_dir, '4_pvalues_eeg_y.csv'))

# variables
y_data.columns = save_index
y_data.to_csv(os.path.join(results_4_dir, '4_variables_eeg_y.csv'))

# group 2
# correlation values
o_correlation_df = pd.DataFrame(data=o_correlation, index=save_index,
                                columns=save_index)
o_correlation_df.to_csv(os.path.join(results_4_dir, '4_correlation_eeg_o.csv'))

# p values
o_pvalue_df = pd.DataFrame(data=o_pvalue, index=save_index,
                           columns=save_index)
o_pvalue_df.to_csv(os.path.join(results_4_dir, '4_pvalues_eeg_o.csv'))

# variables
o_data.columns = save_index
o_data.to_csv(os.path.join(results_4_dir, '4_variables_eeg_o.csv'))